In [1]:
from datasets import load_dataset
docs = load_dataset(f"mhhmm/leetcode-solutions-python", split="train")

Found cached dataset json (C:/Users/Tommaso/.cache/huggingface/datasets/mhhmm___json/mhhmm--leetcode-solutions-python-c6d2758e3a6cc905/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [2]:
docs.column_names

['id', 'code_only', 'code_with_problem', 'code_with_data', 'explanation_only']

In [3]:
docs.shape

(2359, 5)

In [4]:
docs["code_only"][1]

'```python\nclass ListNode:\n    def __init__(self, val=0, next=None):\n        self.val = val\n        self.next = next\n\ndef addTwoNumbers(l1, l2):\n    dummy = ListNode(0)\n    current = dummy\n    carry = 0\n\n    while l1 or l2 or carry:\n        sum_val = (l1.val if l1 else 0) + (l2.val if l2 else 0) + carry\n        carry = sum_val // 10\n        current.next = ListNode(sum_val % 10)\n        current = current.next\n\n        if l1: l1 = l1.next\n        if l2: l2 = l2.next\n\n    return dummy.next\n```\n\n'

In [5]:
import libcst as cst
import re
import random
from openbugger.bugger import Bugger, bugger_example
from time import perf_counter

In [6]:
#import all bugs from openbugger
from openbugger.bugs.controlflow import ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer, gen_OffByKIndexTransformer, IncorrectExceptionHandlerTransformer,MissingArgumentTransformer,ReturningEarlyTransformer
from openbugger.bugs.data import IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer
from openbugger.bugs.logical import gen_ComparisonTargetTransfomer, ComparisonSwapTransformer
from openbugger.bugs.type import IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer
from openbugger.bugs.numpy import NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer

In [7]:
transformers = [gen_OffByKIndexTransformer(1)]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[1]"
bugger = Bugger(transformers)
# Parse the script into a CST
module = cst.parse_module(script)

tainted = bugger.transformers[0].mutate(module)
tainted.code
# clean = bugger.apply(tainted,debug=True

'while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[2]'

In [8]:
code_list = []
problem_list = []
data_list = []
cst_list = []
bad_code = []
explanation_list = []
for code_with_data,code_with_problem,code,explanation in zip(docs["code_with_data"],docs["code_with_problem"],docs["code_only"],docs["explanation_only"]):
    #remove ```python and ``` at the end 
    good_code= False
    clean_code = re.sub(r'```python', '', code)
    clean_code = re.sub(r'```.*', '', clean_code).strip()

    split_data = code_with_data.split('```python')
    split_problem = code_with_problem.split('```python')
    if len(split_data) > 1 and len(split_problem) > 1:
        data = split_data[0]
        problem = split_problem[0]
        
        try:
            cst_tree = cst.parse_module(clean_code)
            good_code = True
            code_list.append(clean_code)
            cst_list.append(cst_tree)
            data_list.append(data)
            problem_list.append(problem)
            explanation_list.append(explanation)
        except:
            bad_code.append(clean_code)

In [9]:
print(len(code_list),len(cst_list),len(data_list),len(problem_list),len(bad_code),len(explanation_list))

2348 2348 2348 2348 2 2348


In [10]:
#create a dictionary where keys are bugs type and values are the bug classes
bug_dict = {"controlflow":[ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer, gen_OffByKIndexTransformer(),IncorrectExceptionHandlerTransformer,MissingArgumentTransformer,ReturningEarlyTransformer],
            "data":[IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer],
            "logical":[gen_ComparisonTargetTransfomer(), ComparisonSwapTransformer],
            "type":[IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer],
            "numpy":[NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer]}
def sample_bug(bug_dict):
    # Select a random bug type
    bug_type = random.choice(list(bug_dict.keys()))
    # Select a random bug from the chosen type
    bug_class = random.choice(bug_dict[bug_type])
    return bug_type, bug_class

In [11]:

#sample a random from code_list and applies bugger_example
random_script = random.sample(code_list,1)[0]

In [12]:
tainted = bugger.transformers[0].mutate(module)

In [13]:
tainted.code

'while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[1]'

In [14]:
bug = """def findMedian(grid: List[List[int]]) -> int:
    m, n = len(grid), len(grid[0])
    low, high = 1, 1_000_000

    while low < high:
        mid = low + (high - low) // 2
        count = sum(bisect_right(row, mid) for row in grid)

        if count < (m * n) // 2 + 1:
            low = mid + 1
        else:
            high = mid

    return low"""

In [15]:
print(tainted.code)


while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[1:2]
x[1]


In [16]:
# print(random_bug[1])
print(script)
bugger = Bugger([ForgettingToUpdateVariableTransformer])
# Parse the script into a CST
module = cst.parse_module(script)

tainted = bugger.transformers[0].mutate(module)
tainted = bugger.position_updater.transform_module(tainted)
clean = bugger.inverse.debug(tainted,bugger.transformers[0].id)
tainted.code

while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[1:2]
x[1]


'while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = y\nx[1:2]\nx[1]'

In [17]:
# for script in code_list:
#     random_bug = IncorrectExceptionHandlerTransformer # sample_bug(bug_dict)
#     transformers=[random_bug]
#     bugger = Bugger(transformers)
#     # Parse the script into a CST
#     module = cst.parse_module(script)
#     tainted = bugger.apply(module)

#     clean = bugger.apply(tainted,debug=True)

KeyboardInterrupt: 

In [22]:
fail = 0
fail_bugs = 0
fail_inversions = 0
bugs = 0
inversions = 0
for script in code_list:
    try:
        random_bug = gen_ComparisonTargetTransfomer() # sample_bug(bug_dict)
        transformers=[random_bug]
        bugger = Bugger(transformers)
        # Parse the script into a CST
        module = cst.parse_module(script)
        tainted = bugger.apply(module)
        if not module.deep_equals(tainted):
            bugs+=1
            try:
                clean = bugger.apply(tainted,debug=True)
                # bugger.print()
                if module.deep_equals(clean):
                    inversions+=1
            except:
                fail_inversions +=1
        
    except:
        fail+=1
        print(fail)
        

In [23]:
print("Bug Name: {name}, fail {fail}, fail_bugs {fail_bugs}, fail_inversions {fail_inversions}, bugs {bugs}, inversions {inversions}, total {total} ".format(name= random_bug.__name__,fail=fail,fail_bugs=fail_bugs,fail_inversions=fail_inversions,bugs=bugs,inversions=inversions, total=len(code_list)))

Bug Name: ReturningEarlyTransformer, fail 0, fail_bugs 0, fail_inversions 0, bugs 2325, inversions 2319, total 2348 


In [ ]:
a

In [ ]:
# tainted_cst_list = []
# inverted_cst_list = []
# time_to_bug = []
# bugger_list = []
# for module in cst_list:
#     start = perf_counter()
#     transformers = [ForgettingToUpdateVariableTransformer]
#     bugger = Bugger(transformers)
#     # Parse the script into a CST
#     tainted = bugger.apply(module)
#     clean = bugger.apply(tainted,debug=True)
#     tainted_cst_list.append(tainted)
#     inverted_cst_list.append(clean)
#     bugger_list.append(bugger)
#     end = perf_counter()
#     time_to_bug.append(end-start)
#     print("Time to bug: ", end - start)


In [ ]:
# id=0
# succesful_bugs_id = []
# failed_inversions = []
# not_bugged_id = []
# for original, tainted, inverted in zip(cst_list, tainted_cst_list, inverted_cst_list):
#     bug_success = not original.deep_equals(tainted)
#     inversion_sucess = original.deep_equals(inverted)
#     if bug_success:
#         succesful_bugs_id.append(id)
#         if not inversion_sucess:
#             failed_inversions.append(id)
#             print(f"something went wrong for the inversion at {id} ".format(id=id))
#     else:
#         not_bugged_id.append(id)
#     id +=1

In [ ]:
# print(len(not_bugged_id),len(succesful_bugs_id),len(failed_inversions))

In [ ]:
from openbugger.bugs.controlflow import ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer, gen_OffByKIndexTransformer
from openbugger.bugs.data import IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer
from openbugger.bugs.logical import gen_ComparisonTargetTransfomer, ComparisonSwapTransformer
from openbugger.bugs.type import IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer
from openbugger.bugs.numpy import NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer
from openbugger.bugger import Bugger
from time import perf_counter

class BuggerAnalyzer:
    def __init__(self):
        self.bug_dict = {
            # "controlflow": [ gen_OffByKIndexTransformer()],
            "data": [IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer],
            "logical": [gen_ComparisonTargetTransfomer(), ComparisonSwapTransformer],
            "type": [IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer],
            "numpy": [NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer]
        }

    def apply_bugs(self, cst_list):
        results = {}

        for bug_type, transformers in self.bug_dict.items():
            results[bug_type] = {}
            for transformer in transformers:
                results[bug_type][transformer.__name__] = {
                    "bugger": None,
                    "tainted": [None]*len(cst_list),
                    "inverted": [None]*len(cst_list),
                    "time_to_bug": 0,
                    "failed_transformations": [],
                    "failed_inversions": [],
                }
                start = perf_counter()
                bugger = Bugger([transformer])
                for idx, module in enumerate(cst_list):
                    try:
                        tainted = bugger.apply(module)
                        results[bug_type][transformer.__name__]["tainted"][idx] = tainted
                        try:
                            clean = bugger.apply(tainted, debug=True)
                            results[bug_type][transformer.__name__]["inverted"][idx] = clean
                        except Exception as e:
                            print(f"Problem occurred during inversion with {transformer.__name__} at index {idx} ")
                            results[bug_type][transformer.__name__]["failed_inversions"].append((idx, str(e)))

                    except Exception as e:
                        print(f"Problem occurred during bugging with {transformer.__name__} at index {idx} ")
                        results[bug_type][transformer.__name__]["failed_transformations"].append((idx, str(e)))
                    
                end = perf_counter()
                results[bug_type][transformer.__name__]["time_to_bug"] = end-start
                print(f"Time to bug for {transformer.__name__}: ", end - start)

        return results


    def check_inversions(self, results, cst_list):
        for bug_type, transformer_dict in results.items():
            for transformer_name, data in transformer_dict.items():
                successful_bugs_id = []
                failed_inversions = []
                not_bugged_id = []
                failed_transformations = []

                for id, (original, tainted, inverted) in zip(range(len(cst_list)), cst_list, data["tainted"], data["inverted"]):
                    if tainted is not None and inverted is not None:
                        bug_success = not original.deep_equals(tainted)
                        inversion_success = original.deep_equals(inverted)
                        if bug_success:
                            successful_bugs_id.append(id)
                            if not inversion_success:
                                failed_inversions.append(id)
                                print(f"Something went wrong for the inversion at {id} for {transformer_name}")
                        else:
                            not_bugged_id.append(id)
                    else:
                        print(f"Skipping inversion check for {transformer_name} at {id} due to failed transformation")
                        failed_transformations.append(id)

                data["successful_bugs_id"] = successful_bugs_id
                data["failed_inversions"] = failed_inversions
                data["not_bugged_id"] = not_bugged_id
                data["failed_transformations"] = failed_transformations

        return results


In [ ]:
analyzer = BuggerAnalyzer()
results = analyzer.apply_bugs(cst_list)


Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 16 
Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 70 
Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 82 
Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 96 
Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 98 
Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 99 
Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 130 
Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 132 
Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 133 
Problem occurred during inversion with IncorrectVariableInitializationTransformer at index 134 
Problem occurred during inversion with Incorre

KeyboardInterrupt: 

In [ ]:
results = analyzer.check_inversions(results, cst_list)
